In [7]:
import numpy as np
import pandas as pd
import itertools
from sklearn.linear_model import LinearRegression; model = LinearRegression()
from sklearn.feature_selection import f_regression
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm
from statsmodels.formula.api import ols

In [2]:
df = pd.read_excel("SalesData.xlsx")
df

,TV Ad Spending x1,Radio Ad Spending x2,Newspaper Ad Spending x3,Website Visits x4,Product Reviews x5,Sales y
0,10000,5000,2000,500,400,15000
1,12000,6000,2500,600,450,17000
2,9000,4000,1500,700,300,14000
3,11000,5000,2000,800,350,16000
4,8000,3000,1000,400,200,13000
5,11000,6000,1500,700,350,16500
6,12000,7000,2000,800,400,17500
7,10000,5000,1500,500,300,15000
8,9000,4000,1000,400,200,14000
9,11000,5000,1500,700,350,16000


In [3]:
new_columns = {
    "TV Ad Spending x1":"x1",
    "Radio Ad Spending x2":"x2",
    "Newspaper Ad Spending x3":"x3",
    "Website Visits x4":"x4",
    "Product Reviews x5":"x5",
    "Sales y":"y",
}
df = df.rename(columns=new_columns)
y = df.y
df = df.drop("y", axis=1)

In [4]:
results =  []
for i in range(len(df.columns)):
    for combo in itertools.combinations(df.columns, i+1):
        X = df[list(combo)]
        model.fit(X, y)
        r2 = model.score(X, y)
        adj_r2 = 1 - (1-model.score(X, y))*(len(y)-1)/(len(y)-X.shape[1]-1)
        f,_ = f_regression(X, y)
        
        n = len(y)
        kf = KFold(n_splits=n)
        press_values = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = y.iloc[train_index], y.iloc[test_index]
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)
            press_values.append(mean_squared_error(y_test, y_pred))
        press = sum(press_values)

        results.append((list(combo), r2, adj_r2, f[0],press))

        results_df = pd.DataFrame(results, columns=[
    "Columns", "r^2", "adjusted_r^2","F","PRESS"
]).sort_values("adjusted_r^2", ascending=False).reset_index(drop=True)

In [5]:
results_df

,Columns,r^2,adjusted_r^2,F,PRESS
0,"[x1, x2, x3, x4]",0.996940,0.994492,478.864000,1.943418e+05
1,"[x1, x2, x3]",0.995819,0.993728,478.864000,1.860354e+05
2,"[x1, x2, x4, x5]",0.996445,0.993601,478.864000,1.903318e+05
3,"[x1, x2, x3, x4, x5]",0.996944,0.993123,478.864000,3.511461e+05
4,"[x1, x2, x5]",0.995058,0.992587,478.864000,2.107821e+05
5,"[x1, x2, x3, x5]",0.995823,0.992481,478.864000,4.001724e+05
6,"[x1, x2, x4]",0.994937,0.992405,478.864000,2.455793e+05
7,"[x1, x2]",0.993496,0.991637,478.864000,2.361517e+05
8,"[x1, x3]",0.987662,0.984137,478.864000,4.816967e+05
9,"[x1, x3, x5]",0.988555,0.982833,478.864000,5.725488e+05


In [12]:
import pandas as pd
import statsmodels.api as sm
from itertools import combinations

# Create a toy dataset
data = {'x1': [1, 2, 3, 4], 'x2': [5, 6, 7, 8], 'x3': [9, 10, 11, 12], 'x4': [13,14,15,16], 'x5':[17,18,19,20], 'y': [20, 30, 40, 50]}
df = pd.DataFrame(data)

# Get all combinations of independent variables
variables = list(df.columns[:-1])
combos = []
for i in range(1, len(variables)+1):
    combos += list(combinations(variables, i))

# Run ANOVA for each combination of variables
for combo in combos:
    formula = 'y ~ ' + ' + '.join(combo)
    model = sm.formula.ols(formula, data=df).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print('Variables:', combo)
    print(anova_table)
    print('\n')

C:\Users\Hpcc\anaconda3\lib\site-packages\statsmodels\base\model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 1, but rank is 0
  warnings.warn('covariance of constraints does not have full '
C:\Users\Hpcc\anaconda3\lib\site-packages\statsmodels\base\model.py:1900: RuntimeWarning: invalid value encountered in true_divide
  F /= J


Variables: ('x1',)
          sum_sq   df   F  PR(>F)
x1           NaN  1.0 NaN     NaN
Residual     0.0  2.0 NaN     NaN


Variables: ('x2',)
                sum_sq   df             F        PR(>F)
x2        5.000000e+02  1.0  1.237940e+30  8.077936e-31
Residual  8.077936e-28  2.0           NaN           NaN


Variables: ('x3',)
                sum_sq   df             F        PR(>F)
x3        5.000000e+02  1.0  3.094850e+29  3.231174e-30
Residual  3.231174e-27  2.0           NaN           NaN


Variables: ('x4',)
                sum_sq   df             F        PR(>F)
x4        5.000000e+02  1.0  3.438722e+28  2.908057e-29
Residual  2.908057e-26  2.0           NaN           NaN


Variables: ('x5',)
                sum_sq   df             F        PR(>F)
x5        5.000000e+02  1.0  3.438722e+28  2.908057e-29
Residual  2.908057e-26  2.0           NaN           NaN


Variables: ('x1', 'x2')
                sum_sq   df             F        PR(>F)
x1        1.128171e+02  1.0  4.469145e+29

Each ANOVA table displays the sum of squares, degrees of freedom, F statistic, and p-value for each independent variable and the residual.

The first ANOVA table displays results for only one independent variable 'x1'. The sum of squares is missing for this variable, which suggests that this variable is not contributing to the model. The second, third, fourth, and fifth ANOVA tables display results for independent variables 'x2', 'x3', 'x4', and 'x5' respectively. In each case, the sum of squares is non-zero, suggesting that each variable is contributing to the model.

The next six ANOVA tables display results for two independent variables. For example, the sixth ANOVA table displays results for independent variables 'x1' and 'x2'. The sum of squares is non-zero for both variables, suggesting that both variables are contributing to the model. The final ANOVA table displays results for all five independent variables. The sum of squares is non-zero for all variables, suggesting that all variables are contributing to the model.

The F statistic is used to determine whether the independent variable(s) have a statistically significant effect on the dependent variable. In general, a larger F statistic and a smaller p-value (i.e., a p-value less than the significance level, often 0.05) indicate a stronger relationship between the independent variable(s) and the dependent variable.

It's difficult to say which variable combination is most significant based only on the ANOVA table. However, we can look at the p-values in the table to get an idea of which variables are more significant than others.

A low p-value indicates that a variable or combination of variables has a significant effect on the dependent variable. From the ANOVA table, we can see that the p-values for all of the individual variables (x1, x2, x3, x4, and x5) are less than 0.05, indicating that they are all significant.

Among the combinations, the ones with the lowest p-values are ('x1', 'x3'), ('x2', 'x5'), and ('x2', 'x4'), all of which have p-values less than 0.000001. However, we should note that the p-values depend on the significance level we choose, and different levels may lead to different conclusions. Additionally, the significance of the variable combinations may also depend on the specific context and goals of the analysis.